# 긍정 / 부정 댓글 분석

## 1. 라이브러리 추가

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split

In [2]:
# 경고창 무시
import warnings
warnings.filterwarnings(action='ignore')

## 2. 데이터 불러오기 및 전처리

In [3]:
train_df = pd.read_csv('../data_list/train_df2.csv', encoding='utf-8')
train_df[:3]

,review_score,review_content,label
0,5,배송 빠르고 굿,1.0
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0.0
2,5,아주 좋아요 바지 정말 좋아서개 더 구매했어요 이 가격에 대박입니다 바느질이 조금 ...,1.0


In [4]:
train_df = train_df[train_df['review_content'].notnull()]
train_df['review_content'] = train_df['review_content'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', x))

In [5]:
test_df = pd.read_csv('../data_list/test_df2.csv', encoding='utf-8')
test_df[:3]

,review_score,review_content,label
0,4.5,색감이 곱네요,1.0
1,5.0,색이 오묘한게 정말 예뻐요 여름에 진짜 잘 입을 것 같아용 케키케,1.0
2,4.5,우선 옷은 되게 얇아요 그리고 제가 키가 큰 편이라 너무 짧고 작을까봐 걱정했는데 ...,1.0


In [6]:
test_df = test_df[test_df['review_content'].notnull()]
test_df['review_content'] = test_df['review_content'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', x))

In [7]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords = sum(stopwords, [])
stopwords[:10]

['휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해']

## 3. tf-idf --> fit_transform

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

In [9]:
okt = Okt()

In [10]:
tfidf = TfidfVectorizer(tokenizer = okt.morphs,
                        ngram_range=(1, 2),
                        min_df = 5,
                        max_df = 0.9,
                        stop_words = stopwords)
tfidf

TfidfVectorizer(max_df=0.9, min_df=5, ngram_range=(1, 2),
                stop_words=['휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희',
                            '따라', '의해', '을', '를', '에', '의', '가', '으로', '로',
                            '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면',
                            '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', ...],
                tokenizer=<bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x0000020876EE6400>>)

In [11]:
tfidf.fit(train_df['review_content'])

TfidfVectorizer(max_df=0.9, min_df=5, ngram_range=(1, 2),
                stop_words=['휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희',
                            '따라', '의해', '을', '를', '에', '의', '가', '으로', '로',
                            '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면',
                            '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', ...],
                tokenizer=<bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x0000020876EE6400>>)

In [12]:
tfidf.vocabulary_.items()

dict_items([('배송', 1462), ('빠르고', 1755), ('굿', 337), ('배송 빠르고', 1474), ('택배', 3591), ('엉망', 2551), ('이네', 2803), ('용', 2706), ('집', 3396), ('밑', 1387), ('층', 3519), ('말', 1164), ('도', 842), ('없이', 2543), ('엉망 이네', 2552), ('말 도', 1165), ('도 없이', 882), ('아주', 2336), ('좋아요', 3246), ('바지', 1399), ('정말', 3155), ('좋아서', 3241), ('개', 128), ('더', 821), ('구매', 287), ('가격', 29), ('대박', 805), ('입니다', 2905), ('바느질', 1390), ('엉', 2549), ('편하고', 3682), ('가성', 55), ('비', 1727), ('최고', 3489), ('예요', 2616), ('아주 좋아요', 2342), ('더 구매', 822), ('가성 비', 56), ('비 최고', 1731), ('최고 예요', 3491), ('선물', 1989), ('빨리', 1792), ('받아서', 1430), ('전달', 3123), ('하는', 3762), ('상품', 1925), ('이었는데', 2841), ('만', 1131), ('와서', 2654), ('당황', 798), ('했습니다', 3945), ('했지만', 3950), ('누락', 665), ('확인', 3980), ('안', 2360), ('하고', 3743), ('날', 539), ('이렇게', 2818), ('오래', 2626), ('다시', 745), ('생각', 1953), ('같아요', 114), ('아쉽네요', 2326), ('선물 용', 1991), ('배송 도', 1470), ('확인 안', 3982), ('안 하고', 2382), ('색상', 1946), ('예뻐요', 2608), ('옆', 2

In [13]:
train_tfidf_df = tfidf.transform(train_df['review_content'])
train_tfidf_df

<8000x4032 sparse matrix of type '<class 'numpy.float64'>'
	with 87444 stored elements in Compressed Sparse Row format>

In [14]:
test_tfidf_df = tfidf.transform(test_df['review_content'])
test_tfidf_df

<11x4032 sparse matrix of type '<class 'numpy.float64'>'
	with 149 stored elements in Compressed Sparse Row format>

In [15]:
tfidf.inverse_transform(train_tfidf_df)

[array(['빠르고', '배송 빠르고', '배송', '굿'], dtype='<U9'),
 array(['택배', '층', '집', '이네', '용', '엉망 이네', '엉망', '없이', '밑', '말 도', '말',
        '도 없이', '도'], dtype='<U9'),
 array(['편하고', '최고 예요', '최고', '좋아요', '좋아서', '정말', '입니다', '예요', '엉',
        '아주 좋아요', '아주', '비 최고', '비', '바지', '바느질', '더 구매', '더', '대박', '구매',
        '개', '가성 비', '가성', '가격'], dtype='<U9'),
 array(['확인 안', '확인', '했지만', '했습니다', '하는', '하고', '전달', '이었는데', '이렇게', '용',
        '와서', '오래', '안 하고', '안', '아쉽네요', '선물 용', '선물', '생각', '상품', '빨리',
        '배송 도', '배송', '받아서', '만', '도', '당황', '다시', '누락', '날', '같아요'],
       dtype='<U9'),
 array(['용', '예뻐요', '옆', '손잡이', '색상', '사용', '되네요', '도 사용', '도', '는', '거 는',
        '거', 'ㅎㅎ'], dtype='<U9'),
 array(['합니다', '코팅', '진짜 별로', '진짜', '적', '입니다', '완전', '예쁘고', '엔', '실용 적',
        '실용', '생각 보다', '생각', '비추 합니다', '비추', '불편해요', '보다', '보기', '별로 입니다',
        '별로', '도', '계란', 'ㅠㅠ'], dtype='<U9'),
 array(['회사', '추천', '주문', '전화 도', '전화', '은', '왔네요', '연락', '없으니', '안되고',
        '아무런', '시켰는데', '분 은', '분'

In [16]:
tfidf.inverse_transform(test_tfidf_df)

[array(['색감'], dtype='<U9'),
 array(['진짜', '정말', '잘', '입', '예뻐요', '여름', '색', '같아용'], dtype='<U9'),
 array(['했는데', '합니다', '할 듯', '할', '하이', '편이 라', '편이', '키', '큰', '짧고', '입고',
        '은', '옷', '얇아요', '바지', '라', '듯', '되게', '너무', '걱정 했는데', '걱정'],
       dtype='<U9'),
 array(['예뻐요', '색감', '사이즈 도', '사이즈', '마음 들어요', '마음', '디자인 도', '디자인', '들어요',
        '도 예뻐요', '도 괜찮고', '도', '너무 마음', '너무', '괜찮고'], dtype='<U9'),
 array(['한참', '하려다가', '하고', '핑크색', '파세요', '이라고', '이고', '은', '요', '예쁜데',
        '엄청', '실물', '생겨', '생각 하고', '생각', '색', '상품', '샀는데', '사진', '사세요',
        '분', '보여요', '보다', '보고', '받고', '반품', '못', '만', '때문', '디자인 은', '디자인',
        '두운', '놀랐네요', '나온', '나서', '깜짝', '그냥', '거', '같은', '갈색'], dtype='<U9'),
 array(['후회', '하는', '짱짱', '중', '예뻐요', '없이', '산', '사세요', '늦게', '그냥', '고민',
        '걸'], dtype='<U9'),
 array(['해서', '좋아하는', '은', '스타일', '생각', '사람 은', '사람', '빠르고', '배송 은', '배송'],
       dtype='<U9'),
 array(['편하게', '좋을거', '작아요', '입기', '얇고', '생각 보다', '생각', '보다 더', '보다', '더',
        '같습니다', 'ㅎ

## 4. 분석 정확도 확인하기

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
SA_lr = LogisticRegression(random_state = 0)

In [19]:
SA_lr.fit(train_tfidf_df, train_df['label'])

LogisticRegression(random_state=0)

In [20]:
test_predict = SA_lr.predict(test_tfidf_df)

In [21]:
from sklearn.metrics import accuracy_score

In [22]:
print('후기 분석(긍정/부정) 정확도 : ', round(accuracy_score(test_df['label'], test_predict), 3))

후기 분석(긍정/부정) 정확도 :  0.727


## 5. 후기 긍정/부정 분석하기

In [23]:
st = test_df['review_content'][9]

In [24]:
st2 = re.sub(r'[^ ㄱ-ㅎ|가-힣]+', "", st)
st2

'옷만 보면 이쁨 바지도 청이나 어두운 계열 잘 어울릴 거 같아여'

In [25]:
st_tfidf_df = tfidf.transform([st2])
st_tfidf_df

<1x4032 sparse matrix of type '<class 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [26]:
st2_predict = SA_lr.predict(st_tfidf_df)
st2_predict, st2_predict[0]

(array([0.]), 0.0)

In [27]:
if(st2_predict[0] == 0):
    print(st2, '==> 부정 감정')
else:
    print(st2, '==> 긍정 감정')

옷만 보면 이쁨 바지도 청이나 어두운 계열 잘 어울릴 거 같아여 ==> 부정 감정


In [28]:
st3 = "색이 사진이랑 다르게 벽돌색이네요 별로에요.. 다시는 구매하고싶지 않아요!! 사기도 정도껏 치세요.."

In [29]:
st4 = re.sub(r'[^ ㄱ-ㅎ|가-힣]+', "", st3)
st4

'색이 사진이랑 다르게 벽돌색이네요 별로에요 다시는 구매하고싶지 않아요 사기도 정도껏 치세요'

In [30]:
st_tfidf_df2 = tfidf.transform([st4])
st_tfidf_df2

<1x4032 sparse matrix of type '<class 'numpy.float64'>'
	with 18 stored elements in Compressed Sparse Row format>

In [31]:
st4_predict = SA_lr.predict(st_tfidf_df2)
st4_predict, st2_predict[0]

(array([0.]), 0.0)

In [32]:
if(st4_predict[0] == 0):
    print(st4, '==> 부정 감정')
else:
    print(st4, '==> 긍정 감정')

색이 사진이랑 다르게 벽돌색이네요 별로에요 다시는 구매하고싶지 않아요 사기도 정도껏 치세요 ==> 부정 감정
